In [3]:
!pip install -q mediapipe


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\lchar\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [4]:
!curl -o pose_landmarker_heavy.task https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 19 29.2M   19 5949k    0     0  4230k      0  0:00:07  0:00:01  0:00:06 4243k
100 29.2M  100 29.2M    0     0  15.5M      0  0:00:01  0:00:01 --:--:-- 15.6M


In [71]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from mediapipe.tasks.python.vision.pose_landmarker import PoseLandmarker
from mediapipe.tasks import python as p
from mediapipe.tasks.python import vision
from sklearn.impute import SimpleImputer

In [13]:
modelo = joblib.load("../models/modelo_clasificacion_actividades.pkl")

In [15]:
# Configuración del detector de MediaPipe
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [17]:
def normalizar_coordenadas(df):
    cadera_derecha = df[df['landmark_index'] == 23][['x', 'y', 'z']].mean()
    cadera_izquierda = df[df['landmark_index'] == 24][['x', 'y', 'z']].mean()
    cadera_centro = (cadera_derecha + cadera_izquierda) / 2

    df['x'] -= cadera_centro['x']
    df['y'] -= cadera_centro['y']
    df['z'] -= cadera_centro['z']

    hombro_derecho = df[df['landmark_index'] == 11][['x', 'y', 'z']].mean()
    hombro_izquierdo = df[df['landmark_index'] == 12][['x', 'y', 'z']].mean()
    torso_tamano = np.linalg.norm(hombro_derecho - hombro_izquierdo)

    df['x'] /= torso_tamano
    df['y'] /= torso_tamano
    df['z'] /= torso_tamano

    return df

In [19]:
def filtrar_datos(df):
    # Convertir las columnas x, y, z a tipo float
    df['x'] = df['x'].astype(float)
    df['y'] = df['y'].astype(float)
    df['z'] = df['z'].astype(float)

    # Aplicar el filtro gaussiano
    df['x'] = gaussian_filter1d(df['x'], sigma=2)
    df['y'] = gaussian_filter1d(df['y'], sigma=2)
    df['z'] = gaussian_filter1d(df['z'], sigma=2)

    return df


In [21]:
def calcular_angulo(p1, p2, p3):
    vector1 = p1 - p2
    vector2 = p3 - p2
    cos_theta = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    angulo = np.arccos(np.clip(cos_theta, -1.0, 1.0))
    return np.degrees(angulo)

In [23]:
def extract_landmarks_to_dataframe(detection_result):
    # Crear una lista para almacenar las coordenadas de cada landmark
    landmarks = []
    for pose_landmark in detection_result.pose_landmarks:
        for landmark in pose_landmark:
            landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])

    # Convertir la lista a un DataFrame de pandas
    df_landmarks = pd.DataFrame(landmarks, columns=['x', 'y', 'z', 'visibility'])

    # Agregar una columna de índice para los landmarks
    df_landmarks['landmark_index'] = df_landmarks.index
    return df_landmarks



In [117]:
def generar_caracteristicas(df):
    articulaciones_clave = [11, 12, 13, 14, 15, 16, 23, 24]

    velocidades = []
    angulos_codo_derecho = []
    angulos_codo_izquierdo = []
    angulos_tronco = []

    for articulacion in articulaciones_clave:
        actual = df[df['landmark_index'] == articulacion]
        if len(actual) > 1:
            vel_x = actual['x'].iloc[-1] - actual['x'].iloc[-2]
            vel_y = actual['y'].iloc[-1] - actual['y'].iloc[-2]
            vel_z = actual['z'].iloc[-1] - actual['z'].iloc[-2]
            velocidad = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
            velocidades.append(velocidad)

    hombro_derecho = df[df['landmark_index'] == 11][['x', 'y', 'z']].values
    codo_derecho = df[df['landmark_index'] == 13][['x', 'y', 'z']].values
    muneca_derecha = df[df['landmark_index'] == 15][['x', 'y', 'z']].values

    if hombro_derecho.size > 0 and codo_derecho.size > 0 and muneca_derecha.size > 0:
        angulos_codo_derecho.append(calcular_angulo(hombro_derecho[0], codo_derecho[0], muneca_derecha[0]))

    hombro_izquierdo = df[df['landmark_index'] == 12][['x', 'y', 'z']].values
    codo_izquierdo = df[df['landmark_index'] == 14][['x', 'y', 'z']].values
    muneca_izquierda = df[df['landmark_index'] == 16][['x', 'y', 'z']].values

    if hombro_izquierdo.size > 0 and codo_izquierdo.size > 0 and muneca_izquierda.size > 0:
        angulos_codo_izquierdo.append(calcular_angulo(hombro_izquierdo[0], codo_izquierdo[0], muneca_izquierda[0]))

    if hombro_derecho.size > 0 and hombro_izquierdo.size > 0:
        cadera_centro = (df[df['landmark_index'] == 23][['x', 'y', 'z']].values +
                         df[df['landmark_index'] == 24][['x', 'y', 'z']].values) / 2
        angulos_tronco.append(calcular_angulo(hombro_derecho[0], cadera_centro[0], hombro_izquierdo[0]))

    caracteristicas =  np.array([
        np.mean(velocidades), np.std(velocidades),
        np.mean(angulos_codo_derecho), np.std(angulos_codo_derecho),
        np.mean(angulos_codo_izquierdo), np.std(angulos_codo_izquierdo),
        np.mean(angulos_tronco), np.std(angulos_tronco)
    ])

    print(caracteristicas.shape)

    return caracteristicas

In [125]:
# Inicializar el detector
base_options = p.BaseOptions(model_asset_path='../posemodels/pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# Configuración de MediaPipe (si es necesario)
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Procesamiento de la imagen para obtener landmarks
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    detection_result = detector.detect(mp_image)

    # Verificar si hay puntos clave detectados
    if detection_result.pose_landmarks:
        # Iterar sobre cada pose_landmark (que es un conjunto de puntos clave)
        for landmark in detection_result.pose_landmarks:
            for point in landmark:  # 'landmark' ahora es directamente el punto clave
                x = int(point.x * frame.shape[1])  # Convertir coordenada x al tamaño de la imagen
                y = int(point.y * frame.shape[0])  # Convertir coordenada y al tamaño de la imagen
                # Dibujar los puntos en la imagen (rojo, tamaño 5)
                cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)

        # Dibujar las conexiones entre los puntos (líneas negras)
        #mp_drawing.draw_landmarks(frame, detection_result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Convertir los resultados de los landmarks en un DataFrame para su procesamiento
    df_landmarks = extract_landmarks_to_dataframe(detection_result)
    df_landmarks = normalizar_coordenadas(df_landmarks)
    df_landmarks = filtrar_datos(df_landmarks)
    df_landmarks.fillna(0, inplace=True)

    # Generar las características para la predicción
    caracteristicas = generar_caracteristicas(df_landmarks)

    imputer = SimpleImputer(strategy='constant', fill_value=0)
    caracteristicas = imputer.fit_transform([caracteristicas])
    
    
    prediccion = modelo.predict(caracteristicas)[0]

    # Mostrar la predicción sobre la imagen
    cv2.putText(frame, f'Prediccion: {prediccion}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar la imagen con los puntos y la predicción
    cv2.imshow('Prediccion en tiempo real', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\si

(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         151.26042049   0.         152.49700572
    0.           1.71524643   0.        ]]
(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         142.65008408   0.         144.62616567
    0.           0.86787927   0.        ]]


C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\si

(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         152.68091488   0.         155.1799227
    0.           1.7073963    0.        ]]
(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         137.34164167   0.         142.22993847
    0.           0.40345778   0.        ]]


C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\si

...

(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         143.26680274   0.         146.0922169
    0.           2.99206236   0.        ]]
(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         149.58965202   0.         151.64023735
    0.           3.52899446   0.        ]]


C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\si

(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         152.50607107   0.         154.57862743
    0.           3.04055559   0.        ]]
(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         150.83005262   0.         153.24103681
    0.           2.95130993   0.        ]]
(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         151.34426053   0.         151.87118876
    0.           2.72388787   0.        ]]


C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(8,)
caracteristicas shape (1, 8)
Número de características esperado por el modelo: 8
Características generadas para predicción:
[[  0.           0.         150.99815952   0.         152.24793394
    0.           1.75764373   0.        ]]


In [91]:
cap.release()
cv2.destroyAllWindows()